In [1]:
# !pip install pandas
# !pip install lightgbm
# !pip install sklearn
import pandas as pd
import numpy as np
import lightgbm as lgb
import joblib
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split
import os
from sklearn.model_selection import TimeSeriesSplit

In [2]:
os.chdir('/tf/notebooks/19th_problem/20_forecast_proton')

In [3]:
train_proton = pd.read_csv("train/train_proton.csv")
test_proton = pd.read_csv("test/test_proton.csv")
# test_proton = pd.read_csv("val/val_proton.csv")

train_AC_H1_EPM = pd.read_csv("train/train_AC_H1_EPM.csv")
test_AC_H1_EPM = pd.read_csv("test/test_AC_H1_EPM.csv")
# test_AC_H1_EPM = pd.read_csv("val/val_AC_H1_EPM.csv")

train_xray = pd.read_csv("train/train_xray.csv")
test_xray = pd.read_csv("test/test_xray.csv")
# test_xray = pd.read_csv("val/val_xray.csv")

train_AC_H0_SWE = pd.read_csv("train/train_AC_H0_SWE.csv")
test_AC_H0_SWE = pd.read_csv("test/test_AC_H0_SWE.csv")
# test_AC_H0_SWE = pd.read_csv("val/val_AC_H0_SWE.csv")

In [4]:
# train_xray['ratio'] = (train_xray['xl']/train_xray['xs'])
# train_xray['r2_plus'] = (train_xray['xl']**2+train_xray['xs']**2)

# test_xray['ratio'] = (test_xray['xl']/test_xray['xs'])
# test_xray['r2_plus'] = (test_xray['xl']**2+test_xray['xs']**2)

In [5]:
# for dataframes in [train_proton,test_proton,train_AC_H1_EPM,test_AC_H1_EPM,train_xray,test_xray,train_AC_H0_SWE,test_AC_H0_SWE]:
#     for column in dataframes.columns:
#         dataframes[column] = np.where(dataframes[column]==-100,0,dataframes[column])

In [6]:
train_AC_H1_EPM['time_tag'] = pd.to_datetime(train_AC_H1_EPM['time_tag'].str.slice(0,-1))

train_AC_H0_SWE['time_tag'] = pd.to_datetime(train_AC_H0_SWE['time_tag'].str.slice(0,-1))

train_proton['time_tag'] = pd.to_datetime(train_proton['time_tag'])

train_xray['time_tag'] = pd.to_datetime(train_xray['time_tag'])

test_AC_H1_EPM['time_tag'] = pd.to_datetime(test_AC_H1_EPM['time_tag'].str.slice(0,-1))

test_AC_H0_SWE['time_tag'] = pd.to_datetime(test_AC_H0_SWE['time_tag'].str.slice(0,-1))

test_proton['time_tag'] = pd.to_datetime(test_proton['time_tag'])

test_xray['time_tag'] = pd.to_datetime(test_xray['time_tag'])

In [7]:
train_AC_H1_EPM['time_tag'] = pd.to_datetime(train_AC_H1_EPM['time_tag'], unit='s')

train_AC_H0_SWE['time_tag'] = pd.to_datetime(train_AC_H0_SWE['time_tag'], unit='s')

train_proton['time_tag'] = pd.to_datetime(train_proton['time_tag'], unit='s')

train_xray['time_tag'] = pd.to_datetime(train_xray['time_tag'], unit='s')

test_AC_H1_EPM['time_tag'] = pd.to_datetime(test_AC_H1_EPM['time_tag'], unit='s')

test_AC_H0_SWE['time_tag'] = pd.to_datetime(test_AC_H0_SWE['time_tag'], unit='s')

test_proton['time_tag'] = pd.to_datetime(test_proton['time_tag'], unit='s')

test_xray['time_tag'] = pd.to_datetime(test_xray['time_tag'], unit='s')

In [8]:
len(train_proton)

799488

# train_AC_H1_EPM

In [9]:
train_AC_H1_EPM = train_AC_H1_EPM.reset_index().set_index('time_tag')

In [10]:
for column in ['P1P_.047-.066MEV_IONS_1/(cm**2-s-sr-MeV)',
       'P2P_.066-.114MEV_IONS_1/(cm**2-s-sr-MeV)',
       'P3P_.114-.190MEV_IONS_1/(cm**2-s-sr-MeV)',
       'P4P_.190-.310MEV_IONS_1/(cm**2-s-sr-MeV)',
       'P5P_.310-.580MEV_IONS_1/(cm**2-s-sr-MeV)',
       'P6P_.580-1.05MEV_IONS_1/(cm**2-s-sr-MeV)',
       'P7P_1.05-1.89MEV_IONS_1/(cm**2-s-sr-MeV)',
       'P8P_1.89-4.75MEV_IONS_1/(cm**2-s-sr-MeV)']:
    if column=='P1P_.047-.066MEV_IONS_1/(cm**2-s-sr-MeV)':
        lens = train_AC_H1_EPM[column].resample('5T').size().reset_index(name=column+'_lens')
        mean = train_AC_H1_EPM[column].resample('5T').mean().reset_index(name=column+'_mean')
#         sums = train_AC_H1_EPM[column].resample('5T').sum().reset_index(name=column+'_sum')
        t = pd.merge(lens,mean,on=['time_tag'],how='left')
#         t = pd.merge(t,sums,on=['time_tag'],how='left')
    else:
        mean = train_AC_H1_EPM[column].resample('5T').mean().reset_index(name=column+'_mean')
#         sums = train_AC_H1_EPM[column].resample('5T').sum().reset_index(name=column+'_sum')
#         t = pd.merge(mean,sums,on=['time_tag'],how='left')
        t = mean.copy()

    if column=='P1P_.047-.066MEV_IONS_1/(cm**2-s-sr-MeV)':
        final_AC_H1_EPM = t
    else:
        final_AC_H1_EPM = pd.merge(final_AC_H1_EPM,t,on=['time_tag'],how='left')

In [11]:
real_train = pd.merge(train_proton,final_AC_H1_EPM,on=['time_tag'],how='left')

In [12]:
test_AC_H1_EPM = test_AC_H1_EPM.reset_index().set_index('time_tag')

In [13]:
for column in ['P1P_.047-.066MEV_IONS_1/(cm**2-s-sr-MeV)',
       'P2P_.066-.114MEV_IONS_1/(cm**2-s-sr-MeV)',
       'P3P_.114-.190MEV_IONS_1/(cm**2-s-sr-MeV)',
       'P4P_.190-.310MEV_IONS_1/(cm**2-s-sr-MeV)',
       'P5P_.310-.580MEV_IONS_1/(cm**2-s-sr-MeV)',
       'P6P_.580-1.05MEV_IONS_1/(cm**2-s-sr-MeV)',
       'P7P_1.05-1.89MEV_IONS_1/(cm**2-s-sr-MeV)',
       'P8P_1.89-4.75MEV_IONS_1/(cm**2-s-sr-MeV)']:
    if column=='P1P_.047-.066MEV_IONS_1/(cm**2-s-sr-MeV)':
        lens = test_AC_H1_EPM[column].resample('5T').size().reset_index(name=column+'_lens')
        mean = test_AC_H1_EPM[column].resample('5T').mean().reset_index(name=column+'_mean')
#         sums = test_AC_H1_EPM[column].resample('5T').sum().reset_index(name=column+'_sum')
        t = pd.merge(lens,mean,on=['time_tag'],how='left')
#         t = pd.merge(t,sums,on=['time_tag'],how='left')
    else:
        mean = test_AC_H1_EPM[column].resample('5T').mean().reset_index(name=column+'_mean')
#         sums = test_AC_H1_EPM[column].resample('5T').sum().reset_index(name=column+'_sum')
#         t = pd.merge(mean,sums,on=['time_tag'],how='left')
        t = mean.copy()

    if column=='P1P_.047-.066MEV_IONS_1/(cm**2-s-sr-MeV)':
        final_AC_H1_EPM = t
    else:
        final_AC_H1_EPM = pd.merge(final_AC_H1_EPM,t,on=['time_tag'],how='left')

In [14]:
real_test = pd.merge(test_proton,final_AC_H1_EPM,on=['time_tag'],how='left')

# train_xray

In [15]:
train_xray.columns

Index(['time_tag', 'xs', 'xl'], dtype='object')

In [16]:
train_xray = train_xray.reset_index().set_index('time_tag')

In [17]:
for column in [x for x in train_xray.columns if x!='index']:
    mean = train_xray[column].resample('5T').mean().reset_index(name=column+'_mean')
#     sums = train_xray[column].resample('5T').sum().reset_index(name=column+'_sum')
#     t = pd.merge(mean,sums,on=['time_tag'],how='left')
    t = mean.copy()

    if column=='xs':
        final_xray = t
    else:
        final_xray = pd.merge(final_xray,t,on=['time_tag'],how='left')

In [18]:
real_train = pd.merge(real_train,final_xray,on=['time_tag'],how='left')

In [19]:
test_xray = test_xray.reset_index().set_index('time_tag')

In [20]:
for column in [x for x in train_xray.columns if x!='index']:
#     sums = test_xray[column].resample('5T').sum().reset_index(name=column+'_sum')
    mean = test_xray[column].resample('5T').mean().reset_index(name=column+'_mean')
#     t = pd.merge(mean,sums,on=['time_tag'],how='left')
    t = mean.copy()

    if column=='xs':
        final_xray = t
    else:
        final_xray = pd.merge(final_xray,t,on=['time_tag'],how='left')

In [21]:
real_test = pd.merge(real_test,final_xray,on=['time_tag'],how='left')

# train_AC_H0_SWE

In [22]:
train_AC_H0_SWE = train_AC_H0_SWE.reset_index().set_index('time_tag')

In [23]:
for column in ['H_DENSITY_#/cc','SW_H_SPEED_km/s']:
    if column=='H_DENSITY_#/cc':
        lens = train_AC_H0_SWE[column].resample('5T').size().reset_index(name=column+'_lens')
#         sums = train_AC_H0_SWE[column].resample('5T').sum().reset_index(name=column+'_sum')
        mean = train_AC_H0_SWE[column].resample('5T').mean().reset_index(name=column+'_mean')
        t = pd.merge(lens,mean,on=['time_tag'],how='left')
#         t = pd.merge(t,sums,on=['time_tag'],how='left')
    else:
#         sums = train_AC_H0_SWE[column].resample('5T').sum().reset_index(name=column+'_sum')
        mean = train_AC_H0_SWE[column].resample('5T').mean().reset_index(name=column+'_mean')
#         t = pd.merge(mean,sums,on=['time_tag'],how='left')
        t = mean.copy()

    if column=='H_DENSITY_#/cc':
        final_AC_H0_SWE = t
    else:
        final_AC_H0_SWE = pd.merge(final_AC_H0_SWE,t,on=['time_tag'],how='left')

In [24]:
real_train = pd.merge(real_train,final_AC_H0_SWE,on=['time_tag'],how='left')

In [25]:
test_AC_H0_SWE = test_AC_H0_SWE.reset_index().set_index('time_tag')

In [26]:
for column in ['H_DENSITY_#/cc','SW_H_SPEED_km/s']:
    if column=='H_DENSITY_#/cc':
        lens = test_AC_H0_SWE[column].resample('5T').size().reset_index(name=column+'_lens')
#         sums = test_AC_H0_SWE[column].resample('5T').sum().reset_index(name=column+'_sum')
        mean = test_AC_H0_SWE[column].resample('5T').mean().reset_index(name=column+'_mean')
        t = pd.merge(lens,mean,on=['time_tag'],how='left')
#         t = pd.merge(t,sums,on=['time_tag'],how='left')
    else:
#         sums = test_AC_H0_SWE[column].resample('5T').sum().reset_index(name=column+'_sum')
        mean = test_AC_H0_SWE[column].resample('5T').mean().reset_index(name=column+'_mean')
#         t = pd.merge(mean,sums,on=['time_tag'],how='left')
        t = mean.copy()

    if column=='H_DENSITY_#/cc':
        final_AC_H0_SWE = t
    else:
        final_AC_H0_SWE = pd.merge(final_AC_H0_SWE,t,on=['time_tag'],how='left')

In [27]:
real_test = pd.merge(real_test,final_AC_H0_SWE,on=['time_tag'],how='left')

# lag 변수

In [28]:
mustin_columns = [x for x in real_train.columns if x not in ['time_tag','proton']]

In [29]:
# lag_list=[1,2,3,4,5,288,576]
lag_list = [1,288,2,-288]
# lag_list = lag_list+[-x for x in lag_list]

train

In [30]:
for lag in lag_list:

    real_train = real_train.set_index('time_tag')

    column_list = [x for x in real_train.columns if (x not in ['proton','time']) and ('_lag' not in x)]

    shift_df = real_train[column_list].shift(lag)

    for i in [{x:x+'_lag'+str(lag)} for x in column_list]:
        shift_df = shift_df.rename(columns=i)

    real_train = pd.merge(real_train,shift_df.reset_index(),on='time_tag',how='left')

test

In [31]:
for lag in lag_list:

    real_test = real_test.set_index('time_tag')

    column_list = [x for x in real_test.columns if (x not in ['proton','time']) and ('_lag' not in x)]

    shift_df = real_test[column_list].shift(lag)

    for i in [{x:x+'_lag'+str(lag)} for x in column_list]:
        shift_df = shift_df.rename(columns=i)

    real_test = pd.merge(real_test,shift_df.reset_index(),on='time_tag',how='left')

In [32]:
real_train = real_train.loc[~real_train['proton'].isnull()].reset_index().drop(['index'],axis=1)

time

In [33]:
time = real_train['time_tag'].dt.time.astype(str)
real_train['time'] = time.str.split(':',expand=True)[0].astype(int)*60+time.str.split(':',expand=True)[1].astype(int)

time = real_test['time_tag'].dt.time.astype(str)
real_test['time'] = time.str.split(':',expand=True)[0].astype(int)*60+time.str.split(':',expand=True)[1].astype(int)

In [34]:
del train_proton
del train_AC_H1_EPM
del test_AC_H1_EPM
del train_xray
del test_xray
del train_AC_H0_SWE
del test_AC_H0_SWE
del shift_df

# 필수 컬럼과 논필수 컬럼 나누기

In [35]:
mustin_columns = mustin_columns+[x for x in real_train.columns if 'lag1' in x]+['time']

In [36]:
lag2_columns = [x for x in real_train.columns if ('lag2' in x) and ('lag288' not in x)]
lag288_columns = [x for x in real_train.columns if ('lag288' in x)]
lag_288_columns = [x for x in real_train.columns if ('lag-288' in x)]

In [37]:
columns_option = [lag2_columns,lag288_columns,lag_288_columns]

# stacking_modeling

In [38]:
def rmsle_eval(y_hat, dtrain):
    y = dtrain.get_label()
    
    y = pd.Series(y)
    y = np.where(y<0,0,y)
    
    y_hat = pd.Series(y_hat)
    y_hat = np.where(y_hat<0,0,y_hat)

    return 'myloss', np.mean((np.log(y_hat+1)-np.log(y+1))**2), False

In [39]:
def rmsle(y_hat, dtrain):
    y = dtrain.get_label()
    
    y = pd.Series(y)
    y = np.where(y<0,0,y)
    
    y_hat = pd.Series(y_hat)
    y_hat = np.where(y_hat<0,0,y_hat)
    grad = (1/(y_hat+1))*(np.log(y_hat+1)-np.log(y+1))
    hess = (1/(y_hat+1))**2-(np.log(y_hat+1)-np.log(y+1))/(y_hat+1)**2
    
#     grad = pd.Series(grad)
#     hess = pd.Series(hess)

#     grad = np.where(y<10,grad,np.where(y<100,grad*100,np.where(y<1000,grad*200,np.where(y<10000,grad*1000,grad*6000))))
#     hess = np.where(y<10,hess,np.where(y<100,hess*100,np.where(y<1000,hess*200,np.where(y<10000,hess*1000,hess*6000))))

    return grad,hess

In [40]:
def get_oof_lgbm(params, train_data, test_data, target_data, num_round, early_round, verbose_round, N_SPLITS=5, random_state=0):

    FOLDs=KFold(n_splits=N_SPLITS, shuffle=True,random_state=0)

    oof = np.zeros(len(train_data))
    predictions = np.zeros(len(test_data))

#     features_lgb = list(train_data.columns)
    feature_importance_df = pd.DataFrame()
    iter_list=[]

    for fold_, (trn_idx, val_idx) in enumerate(FOLDs.split(train_data)):
        trn_data = lgb.Dataset(train_data.iloc[trn_idx], label=target_data.iloc[trn_idx])
        val_data = lgb.Dataset(train_data.iloc[val_idx], label=target_data.iloc[val_idx])

        print("LGB " + str(fold_) + "-" * 50)
        num_round = num_round
        clf = lgb.train(params, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=verbose_round,
                        fobj=rmsle,
                        feval = rmsle_eval,
                        early_stopping_rounds = early_round)
        oof[val_idx] = clf.predict(train_data.iloc[val_idx], num_iteration=clf.best_iteration)
        iter_list.append(clf.best_iteration)
#         fold_importance_df = pd.DataFrame()
#         fold_importance_df["feature"] = features_lgb
#         fold_importance_df["importance"] = clf.feature_importance()
#         fold_importance_df["fold"] = fold_ + 1
#         feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        predictions += clf.predict(test_data, num_iteration=clf.best_iteration) / FOLDs.n_splits
        joblib.dump(clf,'/tf/notebooks/19th_problem/lightgbm_model/lgb'+str(fold_)+'.pkl')
    return oof, predictions, np.mean(iter_list)

In [41]:
import lightgbm as lgb
xgb_params={"objective":"regression",
           "metric":"myloss",
           "max_depth":6,
           "min_child_samples":2,
           "alpha":0.08,
           "gamma":0.06,
           "eta":0.04,
           "subsample":0.08,
           "colsample_bytree":0.97,
           "random_state":2020
           }

In [42]:
agg_train={}
agg_test={}

## 0) original_변수

## a) 변수선택

In [45]:
for i in range(len(columns_option)):
    selected_var = [x for x in columns_option if x!=columns_option[i]]
    selected_var = selected_var[0]+selected_var[1]
    X_train = real_train[mustin_columns+selected_var]
    X_test = real_test[mustin_columns+selected_var]
    y_train = real_train['proton']

    a,b,c=get_oof_lgbm(xgb_params, X_train, X_test, y_train, num_round=100000, early_round=400, verbose_round=500, N_SPLITS=5, random_state=0)

    trn_data = lgb.Dataset(X_train.iloc[:500000], label=y_train.iloc[:500000])
    val_data = lgb.Dataset(X_train.iloc[500000:], label=y_train.iloc[500000:])

    clf = lgb.train(xgb_params, trn_data, int(c),fobj=rmsle,feval = rmsle_eval)
    agg_test[i] = b
    agg_train[i] = clf.predict(X_train.iloc[500000:], num_iteration=clf.best_iteration)
    print(np.mean(b))

LGB 0--------------------------------------------------
Training until validation scores don't improve for 400 rounds
[500]	training's myloss: 0.125593	valid_1's myloss: 0.130477


KeyboardInterrupt: 

In [51]:
tr_stacking = pd.DataFrame({'featureset_1':agg_train[0],
              'featureset_2':agg_train[1],
              'featureset_3':agg_train[2],
              'proton':y_train.iloc[500000:]})

In [53]:
ts_stacking = pd.DataFrame({'featureset_1':agg_test[0],
              'featureset_2':agg_test[1],
              'featureset_3':agg_test[2]})

In [55]:
tr_stacking.to_csv("/tf/notebooks/19th_problem/final_submission_files/stacking_train.csv",index=False)
ts_stacking.to_csv("/tf/notebooks/19th_problem/final_submission_files/stacking_test.csv",index=False)

In [56]:
del tr_stacking
del ts_stacking
del agg_test
del agg_train
del a
del b
del c

## c) permutation_importance

In [1]:
column_list = mustin_columns+lag2_columns+lag288_columns

NameError: name 'mustin_columns' is not defined

In [61]:
X_train = real_train[column_list]
X_test = real_test[column_list]
y_train = real_train['proton']

In [62]:
trn_data = lgb.Dataset(X_train.iloc[:500000], label=y_train.iloc[:500000])
val_data = lgb.Dataset(X_train.iloc[500000:], label=y_train.iloc[500000:])

clf = lgb.train(xgb_params, trn_data, 10000, valid_sets = [trn_data, val_data], verbose_eval=500,
                fobj=rmsle,
                feval = rmsle_eval,
                early_stopping_rounds = 300)

Training until validation scores don't improve for 300 rounds
[500]	training's myloss: 0.114835	valid_1's myloss: 0.172316
Early stopping, best iteration is:
[250]	training's myloss: 0.158742	valid_1's myloss: 0.142359


In [64]:
def rmsle_eval1(y_hat, y):
    
    y = pd.Series(y)
    y = np.where(y<0,0,y)
    
    y_hat = pd.Series(y_hat)
    y_hat = np.where(y_hat<0,0,y_hat)

    return 'myloss', np.mean((np.log(y_hat+1)-np.log(y+1))**2), False

In [65]:
rmsle_list=[]
for column in X_train.columns:
    X_train_fake = X_train.iloc[500000:].copy()
    X_train_fake[column] = X_train_fake[column].sample(len(X_train_fake),replace=False,random_state=2020).reset_index(drop=True)
    rmsle_list.append(rmsle_eval1(clf.predict(X_train_fake, num_iteration=clf.best_iteration),y_train.iloc[500000:]))
    print(column)

P1P_.047-.066MEV_IONS_1/(cm**2-s-sr-MeV)_lens
P1P_.047-.066MEV_IONS_1/(cm**2-s-sr-MeV)_mean
P2P_.066-.114MEV_IONS_1/(cm**2-s-sr-MeV)_mean
P3P_.114-.190MEV_IONS_1/(cm**2-s-sr-MeV)_mean
P4P_.190-.310MEV_IONS_1/(cm**2-s-sr-MeV)_mean
P5P_.310-.580MEV_IONS_1/(cm**2-s-sr-MeV)_mean
P6P_.580-1.05MEV_IONS_1/(cm**2-s-sr-MeV)_mean
P7P_1.05-1.89MEV_IONS_1/(cm**2-s-sr-MeV)_mean
P8P_1.89-4.75MEV_IONS_1/(cm**2-s-sr-MeV)_mean
xs_mean
xl_mean
H_DENSITY_#/cc_lens
H_DENSITY_#/cc_mean
SW_H_SPEED_km/s_mean
time
P1P_.047-.066MEV_IONS_1/(cm**2-s-sr-MeV)_lens_lag1
P1P_.047-.066MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag1
P2P_.066-.114MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag1
P3P_.114-.190MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag1
P4P_.190-.310MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag1
P5P_.310-.580MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag1
P6P_.580-1.05MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag1
P7P_1.05-1.89MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag1
P8P_1.89-4.75MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag1
xs_mean_lag1
xl_mean_lag1
H_DENSITY_#/cc_lens

In [67]:
X_train_fake = X_train.iloc[500000:].copy()
rmsle_eval1(clf.predict(X_train_fake, num_iteration=clf.best_iteration),y_train.iloc[500000:])

('myloss', 0.14235926700979437, False)

In [69]:
imp = pd.DataFrame({'rmsle':[x[1] for x in rmsle_list],'col_list':X_train.columns})

In [70]:
imp['rmsle'] = imp['rmsle']-0.14235926700979437

In [72]:
imp.loc[imp['rmsle']<0]['col_list'].tolist()

['P1P_.047-.066MEV_IONS_1/(cm**2-s-sr-MeV)_mean',
 'P3P_.114-.190MEV_IONS_1/(cm**2-s-sr-MeV)_mean',
 'P4P_.190-.310MEV_IONS_1/(cm**2-s-sr-MeV)_mean',
 'P8P_1.89-4.75MEV_IONS_1/(cm**2-s-sr-MeV)_mean',
 'xs_mean',
 'H_DENSITY_#/cc_mean',
 'P2P_.066-.114MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag1',
 'P4P_.190-.310MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag1',
 'xs_mean_lag1',
 'xl_mean_lag1',
 'H_DENSITY_#/cc_mean_lag1',
 'P1P_.047-.066MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag2',
 'P3P_.114-.190MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag2',
 'xs_mean_lag2',
 'time_lag2',
 'P2P_.066-.114MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag288',
 'P4P_.190-.310MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag288',
 'P5P_.310-.580MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag288',
 'xs_mean_lag288',
 'H_DENSITY_#/cc_mean_lag288',
 'time_lag288']

In [73]:
column_list = [x for x in column_list if x not in imp.loc[imp['rmsle']<0]['col_list'].tolist()]

In [76]:
X_train = real_train[column_list]
X_test = real_test[column_list]

y_train = real_train['proton']

In [77]:
import lightgbm as lgb
xgb_params={"objective":"regression",
           "metric":"myloss",
           "max_depth":6,
           "min_child_samples":2,
           "alpha":0.08,
           "gamma":0.06,
           "eta":0.04,
           "subsample":0.08,
           "colsample_bytree":0.97,
           "random_state":2020
           }
a,b,c=get_oof_lgbm(xgb_params, X_train, X_test, y_train, num_round=100000, early_round=400, verbose_round=500, N_SPLITS=5, random_state=0)

LGB 0--------------------------------------------------
Training until validation scores don't improve for 400 rounds
[500]	training's myloss: 0.134333	valid_1's myloss: 0.135307
[1000]	training's myloss: 0.120879	valid_1's myloss: 0.125057
[1500]	training's myloss: 0.117508	valid_1's myloss: 0.124623
Early stopping, best iteration is:
[1247]	training's myloss: 0.1176	valid_1's myloss: 0.123214
LGB 1--------------------------------------------------
Training until validation scores don't improve for 400 rounds
[500]	training's myloss: 0.134941	valid_1's myloss: 0.136919
[1000]	training's myloss: 0.119466	valid_1's myloss: 0.125774
[1500]	training's myloss: 0.115009	valid_1's myloss: 0.123234
[2000]	training's myloss: 0.11052	valid_1's myloss: 0.120745
Early stopping, best iteration is:
[2087]	training's myloss: 0.110024	valid_1's myloss: 0.12033
LGB 2--------------------------------------------------
Training until validation scores don't improve for 400 rounds
[500]	training's myloss:

In [79]:
tr_stacking = pd.read_csv("/tf/notebooks/19th_problem/final_submission_files/stacking_train.csv")
ts_stacking = pd.read_csv("/tf/notebooks/19th_problem/final_submission_files/stacking_test.csv")

In [81]:
ts_stacking['per_imp1']=b

In [82]:
trn_data = lgb.Dataset(X_train.iloc[:500000], label=y_train.iloc[:500000])
val_data = lgb.Dataset(X_train.iloc[500000:], label=y_train.iloc[500000:])

clf = lgb.train(xgb_params, trn_data, int(c),fobj=rmsle,feval = rmsle_eval)

In [83]:
tr_stacking['per_imp1'] = clf.predict(X_train.iloc[500000:], num_iteration=clf.best_iteration)

In [90]:
tr_stacking.to_csv("/tf/notebooks/19th_problem/final_submission_files/stacking_train.csv",index=False)
ts_stacking.to_csv("/tf/notebooks/19th_problem/final_submission_files/stacking_test.csv",index=False)

In [91]:
del tr_stacking
del ts_stacking

## c) 큰 것들 빼는 것

In [98]:
column_list = mustin_columns+lag2_columns+lag288_columns

In [99]:
X_train = real_train[column_list]
X_test = real_test[column_list]

y_train = real_train['proton']

In [100]:
X_train = X_train.loc[real_train['proton']<=2000].reset_index().drop(['index'],axis=1)
y_train = y_train.loc[real_train['proton']<=2000].reset_index().drop(['index'],axis=1)

In [89]:
import lightgbm as lgb
xgb_params={"objective":"regression",
           "metric":"myloss",
           "max_depth":6,
           "min_child_samples":2,
           "alpha":0.08,
           "gamma":0.06,
           "eta":0.04,
           "subsample":0.08,
           "colsample_bytree":0.97,
           "random_state":2020
           }
a,b,c=get_oof_lgbm(xgb_params, X_train, X_test, y_train, num_round=100000, early_round=400, verbose_round=500, N_SPLITS=5, random_state=0)

LGB 0--------------------------------------------------
Training until validation scores don't improve for 400 rounds
[500]	training's myloss: 0.122609	valid_1's myloss: 0.127126
[1000]	training's myloss: 0.112584	valid_1's myloss: 0.118936
[1500]	training's myloss: 0.109163	valid_1's myloss: 0.117751
Early stopping, best iteration is:
[1417]	training's myloss: 0.109628	valid_1's myloss: 0.117375
LGB 1--------------------------------------------------
Training until validation scores don't improve for 400 rounds
[500]	training's myloss: 0.123417	valid_1's myloss: 0.12784
[1000]	training's myloss: 0.118772	valid_1's myloss: 0.124852
Early stopping, best iteration is:
[798]	training's myloss: 0.119063	valid_1's myloss: 0.124141
LGB 2--------------------------------------------------
Training until validation scores don't improve for 400 rounds
[500]	training's myloss: 0.116595	valid_1's myloss: 0.122054
[1000]	training's myloss: 0.108216	valid_1's myloss: 0.116452
Early stopping, best it

In [101]:
tr_stacking = pd.read_csv("/tf/notebooks/19th_problem/final_submission_files/stacking_train.csv")
ts_stacking = pd.read_csv("/tf/notebooks/19th_problem/final_submission_files/stacking_test.csv")

In [102]:
ts_stacking['big_out2000']=b

In [103]:
trn_data = lgb.Dataset(X_train.iloc[:500000].loc[real_train['proton'].iloc[:500000]<=2000].reset_index().drop(['index'],axis=1),
                       label=y_train.iloc[:500000].loc[real_train['proton'].iloc[:500000]<=2000].reset_index().drop(['index'],axis=1))
val_data = lgb.Dataset(X_train.iloc[500000:], label=y_train.iloc[500000:])

clf = lgb.train(xgb_params, trn_data, int(c),fobj=rmsle,feval = rmsle_eval)

In [107]:
X_train = real_train[column_list]

In [108]:
tr_stacking['big_out2000'] = clf.predict(X_train.iloc[500000:], num_iteration=clf.best_iteration)

In [120]:
tr_stacking.to_csv("/tf/notebooks/19th_problem/final_submission_files/stacking_train.csv",index=False)
ts_stacking.to_csv("/tf/notebooks/19th_problem/final_submission_files/stacking_test.csv",index=False)

In [33]:
# test_proton['proton'] = (ts_stacking['featureset_1']+ts_stacking['featureset_2']+ts_stacking['featureset_3']+ts_stacking['per_imp1']+ts_stacking['big_out2000'])/5

# test_proton['proton'] = np.where(test_proton['proton']<0,0,test_proton['proton'])

# test_proton.to_csv("/tf/notebooks/19th_problem/final_submission_files/5_mean.csv",index=False)

# from aifactory.modules import activate, submit
# submit(19, '/tf/notebooks/19th_problem/산식1.ipynb', '/tf/notebooks/19th_problem/lightgbm_model.zip', '/tf/notebooks/19th_problem/final_submission_files/5_mean.csv')

In [131]:
del ts_stacking
del tr_stacking

## d) orignal

In [34]:
lag2_lag1day_time_col_perimp = ['P1P_.047-.066MEV_IONS_1/(cm**2-s-sr-MeV)_lens',
 'P2P_.066-.114MEV_IONS_1/(cm**2-s-sr-MeV)_mean',
 'P5P_.310-.580MEV_IONS_1/(cm**2-s-sr-MeV)_mean',
 'P6P_.580-1.05MEV_IONS_1/(cm**2-s-sr-MeV)_mean',
 'P7P_1.05-1.89MEV_IONS_1/(cm**2-s-sr-MeV)_mean',
 'xl_mean',
 'H_DENSITY_#/cc_lens',
 'SW_H_SPEED_km/s_mean',
 'P1P_.047-.066MEV_IONS_1/(cm**2-s-sr-MeV)_lens_lag1',
 'P1P_.047-.066MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag1',
 'P3P_.114-.190MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag1',
 'P5P_.310-.580MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag1',
 'P6P_.580-1.05MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag1',
 'P7P_1.05-1.89MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag1',
 'P8P_1.89-4.75MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag1',
 'H_DENSITY_#/cc_lens_lag1',
 'SW_H_SPEED_km/s_mean_lag1',
 'P1P_.047-.066MEV_IONS_1/(cm**2-s-sr-MeV)_lens_lag288',
 'P1P_.047-.066MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag288',
 'P3P_.114-.190MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag288',
 'P6P_.580-1.05MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag288',
 'P7P_1.05-1.89MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag288',
 'P8P_1.89-4.75MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag288',
 'xl_mean_lag288',
 'H_DENSITY_#/cc_lens_lag288',
 'SW_H_SPEED_km/s_mean_lag288',
 'P1P_.047-.066MEV_IONS_1/(cm**2-s-sr-MeV)_lens_lag2',
 'P2P_.066-.114MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag2',
 'P4P_.190-.310MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag2',
 'P5P_.310-.580MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag2',
 'P6P_.580-1.05MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag2',
 'P7P_1.05-1.89MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag2',
 'P8P_1.89-4.75MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag2',
 'xl_mean_lag2',
 'H_DENSITY_#/cc_lens_lag2',
 'H_DENSITY_#/cc_mean_lag2',
 'SW_H_SPEED_km/s_mean_lag2',
 'time']

lag2_lag1day_time_col = ['P1P_.047-.066MEV_IONS_1/(cm**2-s-sr-MeV)_lens',
 'P1P_.047-.066MEV_IONS_1/(cm**2-s-sr-MeV)_mean',
 'P2P_.066-.114MEV_IONS_1/(cm**2-s-sr-MeV)_mean',
 'P3P_.114-.190MEV_IONS_1/(cm**2-s-sr-MeV)_mean',
 'P4P_.190-.310MEV_IONS_1/(cm**2-s-sr-MeV)_mean',
 'P5P_.310-.580MEV_IONS_1/(cm**2-s-sr-MeV)_mean',
 'P6P_.580-1.05MEV_IONS_1/(cm**2-s-sr-MeV)_mean',
 'P7P_1.05-1.89MEV_IONS_1/(cm**2-s-sr-MeV)_mean',
 'P8P_1.89-4.75MEV_IONS_1/(cm**2-s-sr-MeV)_mean',
 'xs_mean',
 'xl_mean',
 'H_DENSITY_#/cc_lens',
 'H_DENSITY_#/cc_mean',
 'SW_H_SPEED_km/s_mean',
 'P1P_.047-.066MEV_IONS_1/(cm**2-s-sr-MeV)_lens_lag1',
 'P1P_.047-.066MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag1',
 'P2P_.066-.114MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag1',
 'P3P_.114-.190MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag1',
 'P4P_.190-.310MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag1',
 'P5P_.310-.580MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag1',
 'P6P_.580-1.05MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag1',
 'P7P_1.05-1.89MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag1',
 'P8P_1.89-4.75MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag1',
 'xs_mean_lag1',
 'xl_mean_lag1',
 'H_DENSITY_#/cc_lens_lag1',
 'H_DENSITY_#/cc_mean_lag1',
 'SW_H_SPEED_km/s_mean_lag1',
 'P1P_.047-.066MEV_IONS_1/(cm**2-s-sr-MeV)_lens_lag288',
 'P1P_.047-.066MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag288',
 'P2P_.066-.114MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag288',
 'P3P_.114-.190MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag288',
 'P4P_.190-.310MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag288',
 'P5P_.310-.580MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag288',
 'P6P_.580-1.05MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag288',
 'P7P_1.05-1.89MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag288',
 'P8P_1.89-4.75MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag288',
 'xs_mean_lag288',
 'xl_mean_lag288',
 'H_DENSITY_#/cc_lens_lag288',
 'H_DENSITY_#/cc_mean_lag288',
 'SW_H_SPEED_km/s_mean_lag288',
 'P1P_.047-.066MEV_IONS_1/(cm**2-s-sr-MeV)_lens_lag2',
 'P1P_.047-.066MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag2',
 'P2P_.066-.114MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag2',
 'P3P_.114-.190MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag2',
 'P4P_.190-.310MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag2',
 'P5P_.310-.580MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag2',
 'P6P_.580-1.05MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag2',
 'P7P_1.05-1.89MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag2',
 'P8P_1.89-4.75MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag2',
 'xs_mean_lag2',
 'xl_mean_lag2',
 'H_DENSITY_#/cc_lens_lag2',
 'H_DENSITY_#/cc_mean_lag2',
 'SW_H_SPEED_km/s_mean_lag2',
 'time']

lag2_lag1day_time_lagminus1day_col = ['P1P_.047-.066MEV_IONS_1/(cm**2-s-sr-MeV)_lens',
 'P1P_.047-.066MEV_IONS_1/(cm**2-s-sr-MeV)_mean',
 'P2P_.066-.114MEV_IONS_1/(cm**2-s-sr-MeV)_mean',
 'P3P_.114-.190MEV_IONS_1/(cm**2-s-sr-MeV)_mean',
 'P4P_.190-.310MEV_IONS_1/(cm**2-s-sr-MeV)_mean',
 'P5P_.310-.580MEV_IONS_1/(cm**2-s-sr-MeV)_mean',
 'P6P_.580-1.05MEV_IONS_1/(cm**2-s-sr-MeV)_mean',
 'P7P_1.05-1.89MEV_IONS_1/(cm**2-s-sr-MeV)_mean',
 'P8P_1.89-4.75MEV_IONS_1/(cm**2-s-sr-MeV)_mean',
 'xs_mean',
 'xl_mean',
 'H_DENSITY_#/cc_lens',
 'H_DENSITY_#/cc_mean',
 'SW_H_SPEED_km/s_mean',
 'P1P_.047-.066MEV_IONS_1/(cm**2-s-sr-MeV)_lens_lag1',
 'P1P_.047-.066MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag1',
 'P2P_.066-.114MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag1',
 'P3P_.114-.190MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag1',
 'P4P_.190-.310MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag1',
 'P5P_.310-.580MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag1',
 'P6P_.580-1.05MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag1',
 'P7P_1.05-1.89MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag1',
 'P8P_1.89-4.75MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag1',
 'xs_mean_lag1',
 'xl_mean_lag1',
 'H_DENSITY_#/cc_lens_lag1',
 'H_DENSITY_#/cc_mean_lag1',
 'SW_H_SPEED_km/s_mean_lag1',
 'P1P_.047-.066MEV_IONS_1/(cm**2-s-sr-MeV)_lens_lag288',
 'P1P_.047-.066MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag288',
 'P2P_.066-.114MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag288',
 'P3P_.114-.190MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag288',
 'P4P_.190-.310MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag288',
 'P5P_.310-.580MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag288',
 'P6P_.580-1.05MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag288',
 'P7P_1.05-1.89MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag288',
 'P8P_1.89-4.75MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag288',
 'xs_mean_lag288',
 'xl_mean_lag288',
 'H_DENSITY_#/cc_lens_lag288',
 'H_DENSITY_#/cc_mean_lag288',
 'SW_H_SPEED_km/s_mean_lag288',
 'P1P_.047-.066MEV_IONS_1/(cm**2-s-sr-MeV)_lens_lag2',
 'P1P_.047-.066MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag2',
 'P2P_.066-.114MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag2',
 'P3P_.114-.190MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag2',
 'P4P_.190-.310MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag2',
 'P5P_.310-.580MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag2',
 'P6P_.580-1.05MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag2',
 'P7P_1.05-1.89MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag2',
 'P8P_1.89-4.75MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag2',
 'xs_mean_lag2',
 'xl_mean_lag2',
 'H_DENSITY_#/cc_lens_lag2',
 'H_DENSITY_#/cc_mean_lag2',
 'SW_H_SPEED_km/s_mean_lag2',
 'P1P_.047-.066MEV_IONS_1/(cm**2-s-sr-MeV)_lens_lag-288',
 'P1P_.047-.066MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag-288',
 'P2P_.066-.114MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag-288',
 'P3P_.114-.190MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag-288',
 'P4P_.190-.310MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag-288',
 'P5P_.310-.580MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag-288',
 'P6P_.580-1.05MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag-288',
 'P7P_1.05-1.89MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag-288',
 'P8P_1.89-4.75MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag-288',
 'xs_mean_lag-288',
 'xl_mean_lag-288',
 'H_DENSITY_#/cc_lens_lag-288',
 'H_DENSITY_#/cc_mean_lag-288',
 'SW_H_SPEED_km/s_mean_lag-288',
 'time']

In [ ]:
selected_var = [lag2_lag1day_time_col_perimp,lag2_lag1day_time_col,lag2_lag1day_time_lagminus1day_col]

In [ ]:
for i in range(len(selected_var)):
    selected_var = selected_var[i]
    X_train = real_train[selected_var]
    X_test = real_test[selected_var]
    y_train = real_train['proton']

    a,b,c=get_oof_lgbm(xgb_params, X_train, X_test, y_train, num_round=100000, early_round=400, verbose_round=500, N_SPLITS=5, random_state=0)

    trn_data = lgb.Dataset(X_train.iloc[:500000], label=y_train.iloc[:500000])
    val_data = lgb.Dataset(X_train.iloc[500000:], label=y_train.iloc[500000:])

    clf = lgb.train(xgb_params, trn_data, int(c),fobj=rmsle,feval = rmsle_eval)
    agg_test[i] = b
    agg_train[i] = clf.predict(X_train.iloc[500000:], num_iteration=clf.best_iteration)
    print(np.mean(b))

## final) ridge_stacking

In [10]:
from sklearn.linear_model import Ridge

In [11]:
tr_stacking = pd.read_csv("/tf/notebooks/19th_problem/final_submission_files/stacking_train.csv")
ts_stacking = pd.read_csv("/tf/notebooks/19th_problem/final_submission_files/stacking_test.csv")

In [12]:
weight = [0.2,0,0.2,0,0.6]

In [27]:
test_proton['proton'] = weight[0]*ts_stacking['featureset_1']+weight[1]*ts_stacking['featureset_2']+weight[2]*ts_stacking['featureset_3']+weight[3]*ts_stacking['per_imp1']+weight[4]*ts_stacking['big_out2000']

In [29]:
test_proton['proton'] = np.where(test_proton['proton']<0,0,test_proton['proton'])

test_proton.to_csv("/tf/notebooks/19th_problem/final_submission_files/5_weigted_mean20206.csv",index=False)

In [31]:
from aifactory.modules import activate, submit
submit(19, '/tf/notebooks/19th_problem/산식1.ipynb', '/tf/notebooks/19th_problem/lightgbm_model.zip', '/tf/notebooks/19th_problem/final_submission_files/5_weigted_mean20206.csv')

: 200


21.7MB [00:00, 51.5MB/s]                            

업로드 완료 (시각: 2020-06-22 20:30:17.411009): 200


In [169]:
for i in ['featureset_1', 'featureset_2', 'featureset_3', 'per_imp1',
       'big_out2000']:
    print(rmsle_eval1(tr_stacking[i],tr_stacking['proton']))

('myloss', 0.2037714510622045, False)
('myloss', 0.23791584213371628, False)
('myloss', 0.20690735955682354, False)
('myloss', 0.25159800812599087, False)
('myloss', 0.19405866569857635, False)


In [170]:
rmsle_eval1(pred,tr_stacking['proton'])

('myloss', 0.18229087950824288, False)

In [ ]:
tr_stacking

In [135]:
tr_stacking.columns

Index(['featureset_1', 'featureset_2', 'featureset_3', 'proton', 'per_imp1',
       'big_out2000'],
      dtype='object')

In [138]:
X_train = tr_stacking[['featureset_1', 'featureset_2', 'featureset_3', 'per_imp1',
       'big_out2000']]
y_train = tr_stacking['proton']

X_test = ts_stacking[['featureset_1', 'featureset_2', 'featureset_3', 'per_imp1',
       'big_out2000']]

In [146]:
ridge = Ridge(alpha=0).fit(X_train, y_train)

In [147]:
ridge.predict(X_test)

array([-1.79771159, -2.07153666, -1.98054242, ..., -1.65785025,
       -1.6063802 , -1.70603065])

In [119]:
rmsle_eval1((tr_stacking['featureset_1']+tr_stacking['featureset_2']+tr_stacking['featureset_3']+tr_stacking['per_imp1']+tr_stacking['big_out2000'])/5,tr_stacking['proton'])

('myloss', 0.19433199134786605, False)

In [113]:
rmsle_eval1(tr_stacking['featureset_1'],tr_stacking['proton'])

('myloss', 0.2037714510622045, False)

In [115]:
rmsle_eval1(tr_stacking['featureset_2'],tr_stacking['proton'])

('myloss', 0.23791584213371628, False)

In [116]:
rmsle_eval1(tr_stacking['featureset_3'],tr_stacking['proton'])

('myloss', 0.20690735955682354, False)

In [117]:
rmsle_eval1(tr_stacking['per_imp1'],tr_stacking['proton'])

('myloss', 0.2515980081259908, False)

In [118]:
rmsle_eval1(tr_stacking['big_out2000'],tr_stacking['proton'])

('myloss', 0.19405866569857635, False)

## d) lag1_proton

# training

In [49]:
X_train = real_train[[x for x in real_train.columns if x not in ['time_tag','proton']]]
X_test = real_test[[x for x in real_test.columns if x not in ['time_tag','proton']]]

y_train = real_train['proton']

In [95]:
def rmsle_eval(y_hat, dtrain):
    y = dtrain.get_label()
    
    y = pd.Series(y)
    y = np.where(y<0,0,y)
    
    y_hat = pd.Series(y_hat)
    y_hat = np.where(y_hat<0,0,y_hat)

    return 'myloss', np.mean((np.log(y_hat+1)-np.log(y+1))**2), False

In [96]:
def rmsle(y_hat, dtrain):
    y = dtrain.get_label()
    
    y = pd.Series(y)
    y = np.where(y<0,0,y)
    
    y_hat = pd.Series(y_hat)
    y_hat = np.where(y_hat<0,0,y_hat)
    grad = (1/(y_hat+1))*(np.log(y_hat+1)-np.log(y+1))
    hess = (1/(y_hat+1))**2-(np.log(y_hat+1)-np.log(y+1))/(y_hat+1)**2
    
#     grad = pd.Series(grad)
#     hess = pd.Series(hess)

#     grad = np.where(y<10,grad,np.where(y<100,grad*100,np.where(y<1000,grad*200,np.where(y<10000,grad*1000,grad*6000))))
#     hess = np.where(y<10,hess,np.where(y<100,hess*100,np.where(y<1000,hess*200,np.where(y<10000,hess*1000,hess*6000))))

    return grad,hess

In [43]:
import joblib

일반 validation

In [44]:
def get_oof_lgbm(params, train_data, test_data, target_data, num_round, early_round, verbose_round, N_SPLITS=5, random_state=0):

    FOLDs=KFold(n_splits=N_SPLITS, shuffle=True,random_state=0)

    oof = np.zeros(len(train_data))
    predictions = np.zeros(len(test_data))

#     features_lgb = list(train_data.columns)
    feature_importance_df = pd.DataFrame()

    for fold_, (trn_idx, val_idx) in enumerate(FOLDs.split(train_data)):
        trn_data = lgb.Dataset(train_data.iloc[trn_idx], label=target_data.iloc[trn_idx])
        val_data = lgb.Dataset(train_data.iloc[val_idx], label=target_data.iloc[val_idx])

        print("LGB " + str(fold_) + "-" * 50)
        num_round = num_round
        clf = lgb.train(params, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=verbose_round,
                        fobj=rmsle,
                        feval = rmsle_eval,
                        early_stopping_rounds = early_round)
        oof[val_idx] = clf.predict(train_data.iloc[val_idx], num_iteration=clf.best_iteration)

#         fold_importance_df = pd.DataFrame()
#         fold_importance_df["feature"] = features_lgb
#         fold_importance_df["importance"] = clf.feature_importance()
#         fold_importance_df["fold"] = fold_ + 1
#         feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        predictions += clf.predict(test_data, num_iteration=clf.best_iteration) / FOLDs.n_splits
        joblib.dump(clf,'lgb'+str(fold_)+'.pkl')
    return oof, predictions, feature_importance_df

In [45]:
os.chdir('/tf/notebooks/19th_problem/lightgbm_model')

In [46]:
import lightgbm as lgb
xgb_params={"objective":"regression",
           "metric":"myloss",
           "max_depth":6,
           "min_child_samples":2,
           "alpha":0.08,
           "gamma":0.06,
           "eta":0.04,
           "subsample":0.08,
           "colsample_bytree":0.97,
           "random_state":2020
           }
a,b,c=get_oof_lgbm(xgb_params, X_train, X_test, y_train, num_round=100000, early_round=400, verbose_round=500, N_SPLITS=5, random_state=0)

LGB 0--------------------------------------------------
Training until validation scores don't improve for 400 rounds
[500]	training's myloss: 0.129578	valid_1's myloss: 0.133294
[1000]	training's myloss: 0.112044	valid_1's myloss: 0.118906
[1500]	training's myloss: 0.108441	valid_1's myloss: 0.118753
Early stopping, best iteration is:
[1269]	training's myloss: 0.109246	valid_1's myloss: 0.117773
LGB 1--------------------------------------------------
Training until validation scores don't improve for 400 rounds
[500]	training's myloss: 0.130695	valid_1's myloss: 0.134124
[1000]	training's myloss: 0.11843	valid_1's myloss: 0.126736
[1500]	training's myloss: 0.114594	valid_1's myloss: 0.125752
Early stopping, best iteration is:
[1252]	training's myloss: 0.115024	valid_1's myloss: 0.124557
LGB 2--------------------------------------------------
Training until validation scores don't improve for 400 rounds
[500]	training's myloss: 0.131897	valid_1's myloss: 0.13946
[1000]	training's mylos

In [122]:
test_proton['proton'] = b

In [51]:
test_proton['proton'] = np.where(test_proton['proton']<0,0,test_proton['proton'])

test_proton.to_csv("../eval_lag2_lag1day.csv",index=False)

In [52]:
os.chdir('..')

In [53]:
import shutil
zip_name = 'lightgbm_model'
directory_name = 'lightgbm_model'
# Create 'path\to\zip_file.zip'
shutil.make_archive(zip_name, 'zip', directory_name)

'/tf/notebooks/19th_problem/lightgbm_model.zip'

In [80]:
from aifactory.modules import activate, submit
submit(19, '산식1.ipynb', 'lightgbm_model.zip', 'eval_lag2_lag1day.csv')

 15%|█▍        | 3.20M/21.7M [00:00<00:00, 33.3MB/s]

: 200


21.7MB [00:00, 65.5MB/s]                            

업로드 완료 (시각: 2020-06-22 04:57:26.856196): 200


# 변수 중요도

In [97]:
len(X_train)

799488

In [109]:
xgb_params={"objective":"regression",
           "metric":"myloss",
           "max_depth":6,
           "min_child_samples":2,
           "alpha":0.08,
           "gamma":0.06,
           "eta":0.04,
           "subsample":0.08,
           "colsample_bytree":0.97,
           "random_state":2020
           }

In [110]:
trn_data = lgb.Dataset(X_train, label=y_train)
val_data = lgb.Dataset(X_val, label=y_val)

clf = lgb.train(xgb_params, trn_data, 10000, valid_sets = [trn_data, val_data], verbose_eval=500,
                fobj=rmsle,
                feval = rmsle_eval,
                early_stopping_rounds = 300)

Training until validation scores don't improve for 300 rounds
[500]	training's myloss: 0.132788	valid_1's myloss: 0.0594058
Early stopping, best iteration is:
[247]	training's myloss: 0.159696	valid_1's myloss: 0.0534542


In [124]:
test_proton['proton'] = clf.predict(X_test, num_iteration=clf.best_iteration)

In [127]:
test_proton['proton'] = np.where(test_proton['proton']<0,0,test_proton['proton'])

test_proton.to_csv("../eval_lag2_lag1day_time_validationset.csv",index=False)

In [128]:
os.chdir('..')

In [139]:
from aifactory.modules import activate, submit
submit(19, '산식1.ipynb', 'lightgbm_model.zip', 'eval_lag2_lag1day_time_validationset.csv')

 18%|█▊        | 3.98M/21.7M [00:00<00:00, 41.4MB/s]

: 200


21.7MB [00:00, 69.6MB/s]                            

업로드 완료 (시각: 2020-06-22 05:55:46.193671): 200


In [50]:
trn_data = lgb.Dataset(X_train.iloc[:500000], label=y_train.iloc[:500000])
val_data = lgb.Dataset(X_train.iloc[500000:], label=y_train.iloc[500000:])

clf = lgb.train(xgb_params, trn_data, 10000, valid_sets = [trn_data, val_data], verbose_eval=500,
                fobj=rmsle,
                feval = rmsle_eval,
                early_stopping_rounds = 300)

Training until validation scores don't improve for 300 rounds
[500]	training's myloss: 0.115025	valid_1's myloss: 0.176178
Early stopping, best iteration is:
[262]	training's myloss: 0.153925	valid_1's myloss: 0.144744


In [94]:
def rmsle_eval(y_hat, y):
    
    y = pd.Series(y)
    y = np.where(y<0,0,y)
    
    y_hat = pd.Series(y_hat)
    y_hat = np.where(y_hat<0,0,y_hat)

    return 'myloss', np.mean((np.log(y_hat+1)-np.log(y+1))**2), False

In [52]:
rmsle_list=[]
for column in X_train.columns:
    X_train_fake = X_train.iloc[500000:].copy()
    X_train_fake[column] = X_train_fake[column].sample(len(X_train_fake),replace=False,random_state=2020).reset_index(drop=True)
    rmsle_list.append(rmsle_eval(clf.predict(X_train_fake, num_iteration=clf.best_iteration),y_train.iloc[500000:]))
    print(column)

P1P_.047-.066MEV_IONS_1/(cm**2-s-sr-MeV)_lens
P1P_.047-.066MEV_IONS_1/(cm**2-s-sr-MeV)_mean
P2P_.066-.114MEV_IONS_1/(cm**2-s-sr-MeV)_mean
P3P_.114-.190MEV_IONS_1/(cm**2-s-sr-MeV)_mean
P4P_.190-.310MEV_IONS_1/(cm**2-s-sr-MeV)_mean
P5P_.310-.580MEV_IONS_1/(cm**2-s-sr-MeV)_mean
P6P_.580-1.05MEV_IONS_1/(cm**2-s-sr-MeV)_mean
P7P_1.05-1.89MEV_IONS_1/(cm**2-s-sr-MeV)_mean
P8P_1.89-4.75MEV_IONS_1/(cm**2-s-sr-MeV)_mean
xs_mean
xl_mean
H_DENSITY_#/cc_lens
H_DENSITY_#/cc_mean
SW_H_SPEED_km/s_mean
P1P_.047-.066MEV_IONS_1/(cm**2-s-sr-MeV)_lens_lag1
P1P_.047-.066MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag1
P2P_.066-.114MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag1
P3P_.114-.190MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag1
P4P_.190-.310MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag1
P5P_.310-.580MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag1
P6P_.580-1.05MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag1
P7P_1.05-1.89MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag1
P8P_1.89-4.75MEV_IONS_1/(cm**2-s-sr-MeV)_mean_lag1
xs_mean_lag1
xl_mean_lag1
H_DENSITY_#/cc_lens_lag1

('myloss', 0.14474437579789878, False)

In [55]:
X_train_fake = X_train.iloc[500000:].copy()
rmsle_eval(clf.predict(X_train_fake, num_iteration=clf.best_iteration),y_train.iloc[500000:])

imp = pd.DataFrame({'rmsle':[x[1] for x in rmsle_list],'col_list':X_train.columns})

In [56]:
imp['rmsle'] = imp['rmsle']-0.14474437579789878

In [51]:
imp.loc[imp['rmsle']<=0]['col_list'].tolist()

NameError: name 'imp' is not defined

In [82]:
imp.sort_values(['rmsle']).iloc[:30]

,rmsle,col_list
24,-0.009340,P8P_1.89-4.75MEV_IONS_1/(cm**2-s-sr-MeV)_mean_...
5,-0.005038,P5P_.310-.580MEV_IONS_1/(cm**2-s-sr-MeV)_mean
4,-0.003498,P4P_.190-.310MEV_IONS_1/(cm**2-s-sr-MeV)_mean
64,-0.001724,time
26,-0.001424,xl_mean_lag1
50,-0.001129,P2P_.066-.114MEV_IONS_1/(cm**2-s-sr-MeV)_mean_...
43,-0.000997,ratio_mean_lag288
42,-0.000375,xl_mean_lag288
58,-0.000339,xl_mean_lag2
52,-0.000239,P4P_.190-.310MEV_IONS_1/(cm**2-s-sr-MeV)_mean_...


In [ ]:
oof[val_idx] = clf.predict(train_data.iloc[val_idx], num_iteration=clf.best_iteration)
predictions += clf.predict(test_data, num_iteration=clf.best_iteration) / FOLDs.n_splits

# lag변수 proton

In [54]:
real_train['proton_pred'] = a
real_test['proton_pred'] = b

In [51]:
real_train.to_csv("../real_train_with_pred.csv",index=False)
real_test.to_csv("../real_test_with_pred.csv",index=False)

In [53]:
real_train = pd.read_csv("../real_train_with_pred.csv")
real_test = pd.read_csv("../real_test_with_pred.csv")

In [55]:
real_train = real_train.set_index('time_tag')

shift_df = real_train[['proton_pred']].shift(1)

shift_df = shift_df.rename(columns = {'proton_pred':'proton_pred_lag1'})
    
real_train = pd.merge(real_train,shift_df.reset_index(),on='time_tag',how='left')

In [56]:
real_test = real_test.set_index('time_tag')

shift_df = real_test[['proton_pred']].shift(1)

shift_df = shift_df.rename(columns={'proton_pred':'proton_pred_lag1'})
    
real_test = pd.merge(real_test,shift_df.reset_index(),on='time_tag',how='left')

In [57]:
del real_train['proton_pred']
del real_test['proton_pred']

In [58]:
X_train = real_train[[x for x in real_train.columns if x not in ['time_tag','proton']]]
X_test = real_test[[x for x in real_test.columns if x not in ['time_tag','proton']]]

y_train = real_train['proton']

In [59]:
real_train['proton_pred_lag1'].corr(real_train['proton'])

0.7568272912313208

In [60]:
import lightgbm as lgb
xgb_params={"objective":"regression",
           "metric":"myloss",
           "max_depth":6,
           "min_child_samples":2,
           "alpha":0.08,
           "gamma":0.06,
           "eta":0.04,
           "subsample":0.08,
           "colsample_bytree":0.97,
           "random_state":2020
           }
a,b,c=get_oof_lgbm(xgb_params, X_train, X_test, y_train, num_round=100000, early_round=400, verbose_round=500, N_SPLITS=5, random_state=0)

LGB 0--------------------------------------------------
Training until validation scores don't improve for 400 rounds
[500]	training's myloss: 0.0907622	valid_1's myloss: 0.0951908
[1000]	training's myloss: 0.085191	valid_1's myloss: 0.0937214
Early stopping, best iteration is:
[946]	training's myloss: 0.0854912	valid_1's myloss: 0.0933648
LGB 1--------------------------------------------------
Training until validation scores don't improve for 400 rounds
[500]	training's myloss: 0.0891628	valid_1's myloss: 0.0935346
[1000]	training's myloss: 0.085707	valid_1's myloss: 0.0935802
Early stopping, best iteration is:
[668]	training's myloss: 0.0874615	valid_1's myloss: 0.093183
LGB 2--------------------------------------------------
Training until validation scores don't improve for 400 rounds
[500]	training's myloss: 0.0892055	valid_1's myloss: 0.0991674
Early stopping, best iteration is:
[564]	training's myloss: 0.088263	valid_1's myloss: 0.098645
LGB 3-----------------------------------

In [64]:
test_proton['proton'] = b

test_proton['proton'] = np.where(test_proton['proton']<0,0,test_proton['proton'])

In [72]:
test_proton.to_csv("eval_lag2_lag1day_time_xray_lag1proton.csv",index=False)

In [66]:
os.chdir('..')

In [70]:
os.chdir('./19th_problem')

In [73]:
from aifactory.modules import activate, submit
submit(19, '산식1.ipynb', 'lightgbm_model.zip', 'eval_lag2_lag1day_time_xray_lag1proton.csv')

  0%|          | 0.00/21.7M [00:00<?, ?B/s]

: 200


21.7MB [00:00, 28.1MB/s]                            

업로드 완료 (시각: 2020-06-21 22:22:23.687195): 200


In [40]:
# 1. time series cross validation
# 2. 산식 최적화(최적화2, 최적화1, 아무것도 안함(rmse))
# 3. 변수선택